In [72]:
# -*- coding: utf-8 -*-
#  Copyright 2024 -  United Kingdom Research and Innovation
#  Copyright 2024 -  The University of Manchester
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#  Authored by:    Margaret Duff (STFC-UKRI)
#                  Vaggelis Papoutellis (Finden)  
# 

In [ ]:
from cil.optimisation.functions import L2NormSquared, TotalVariation, MixedL21Norm
from cil.optimisation.operators import BlockOperator, FiniteDifferenceOperator, CompositionOperator, DiagonalOperator
from cil.optimisation.algorithms import PDHG
from cil.utilities import dataexample
from cil.plugins.ccpi_regularisation.functions import FGP_dTV
import numpy as np 

from cil.utilities import dataexample
from cil.utilities.display import show2D
from cil.recon import FDK
from cil.processors import TransmissionAbsorptionConverter, Slicer

import numpy as np

from cil.plugins.tigre import ProjectionOperator
from cil.optimisation.algorithms import FISTA
from cil.optimisation.functions import LeastSquares, TotalVariation
from cil.optimisation.operators import  BlurringOperator

import matplotlib.pyplot as plt

# set up default colour map for visualisation
cmap = "gray"

# set the backend for FBP and the ProjectionOperator
device = 'gpu'

import cil
print(cil.__version__)



# Simulated sphere cone beam reconstruction using directional total variation 

In this notebook we motivate and demonstrate the directional TV regulariser from the CCPi regularisation toolkit. We use this regulariser to reconstruct a slice from a simulated sphere cone beam dataset where the forward model involves a gaussian blur and then a tomographic cone beam projection. We then show how this regulariser could be implemented using the CIL optimisation toolkit. 

## Load, create and show the data

We use the simulated spheres dataset contained in the CIL `dataexample` module. We consider just the central slice for this notebook. 

In [ ]:
#%% Load data
ground_truth = dataexample.SIMULATED_SPHERE_VOLUME.get()

data = dataexample.SIMULATED_CONE_BEAM_DATA.get()

data = data.get_slice(vertical='centre')
ag = data.geometry
absorption = TransmissionAbsorptionConverter()(data)

ground_truth = ground_truth.get_slice(vertical='centre')
ig = ground_truth.geometry

show2D([ground_truth], title = ['Ground Truth'], origin = 'upper', num_cols = 1)


For the forward model, we will innclude both a tomographic projection and a blurr. In the next cell, we define a point spread function and pass this to the CIL `BlurringOperator`.

In [75]:
def psf(n=5, sig=1.):
    """
    creates psf with side length `n` and a sigma of `sig`
    """
    ax = np.linspace(-(n - 1) / 2., (n - 1) / 2., n)
    gauss = np.exp(-0.5 * np.square(ax) / np.square(sig))
    kernel = np.outer(gauss, gauss)
    return kernel / np.sum(kernel)
PSF = psf(5,2)
G = BlurringOperator(PSF,ig )

We now define the tomographic projection operator. 

In [76]:
A = ProjectionOperator(image_geometry=ig, 
                       acquisition_geometry=ag)

We now compose the two opertors and apply it to our ground truth data before adding noise. We compare the original sinogram with the new noisy and blurred sinogram. 

In [ ]:

forward=CompositionOperator(A, G)

noisy_absorption=forward.direct(ground_truth) 
noisy_absorption+= 0.1*noisy_absorption.array.max()*noisy_absorption.geometry.allocate('random')

show2D([absorption, noisy_absorption], title = ['Absorption', 'Noisy absorption'], origin = 'upper', num_cols = 2)




We use the tomography forward model (ignoring the blurring operator) to do an FDK reconstruction on this data. As expected, it is both blurred and noisy compared to the ground truth image. In the next few sections, we will try and improve this reconstruction.


In [ ]:
recon_FDK = FDK(noisy_absorption, image_geometry=ig).run()

show2D([ground_truth, recon_FDK], title = ['Ground Truth', 'FDK Reconstruction'], origin = 'upper', num_cols = 2)

## FISTA + TV Recon 

As a baseline we try a TV reconstruction with a non-negativity constraint, using the FISTA algorithm, minimising the objective 

$$ \arg \min_x \|AGx-y\|_2^2  + \alpha g(x)$$ 

 where $G$ is the blurring operator, $A$ the tomographic projection, $ y$ the noisy measured data and $g$ is the TV regulariser with regularisation paramater $\alpha$. 

 We try a few regularisation parameters and visualise the results: 

In [ ]:
for alpha in [1, 10, 100, 200]:
    F = LeastSquares(A = forward, b = noisy_absorption)
    G = alpha*TotalVariation(lower=0)

    algo_tv=FISTA(initial=ig.allocate(0), f=F, g=G, update_objective_interval=10) 
    algo_tv.run(250)
    show2D([ground_truth, recon_FDK, algo_tv.solution], title = ['Ground Truth', 'FDK Reconstruction', 'TV solution, alpha = {}'.format(alpha)], origin = 'upper', num_cols = 3, fix_range=(0,0.004))
    show2D([ground_truth, recon_FDK - ground_truth, algo_tv.solution - ground_truth], title = ['Ground Truth', 'FDK reconstruction error', 'TV solution error, alpha = {}'.format(alpha)], origin = 'upper', num_cols = 3, fix_range=[(0,0.004),(-0.004,0.004),(-0.004,0.004)], cmap=['gray', 'seismic', 'seismic'])

 

## Directional Total Variation 

The directional total variation regulariser uses a reference image. The idea is that the $dTV(x)$ function is small if all the gradients of $x$ are parallel to the gradients of a reference image. 

We definethe directional total variation regulariser

$$g(x) =dTV(x):= \sum_i|D_i\nabla x_i|_2$$

 where the sum is over the pixels $i$ and where $D$ is a weighting vector filed on the gradient in $x$ dependent on the normalised gradient, $\zeta$,  of the reference image, $\nu$ so 
$$D=I-\zeta \zeta^T$$
and $$\zeta = -\dfrac{\nabla \nu }{\sqrt{\eta^2+|\nabla\nu|^2}}$$ where $0<\eta<<\|\nabla\nu\|$.


We can see that if $\nabla x= \gamma \nabla \nu$, the gradients of $x$ are a multiple of the gradients of the referece image,  then

 $$D\nabla x = \gamma D\nabla \nu= \gamma (I-\zeta \zeta^T)\nabla \nu= \gamma \left(\nabla \nu -\dfrac{\nabla \nu }{\sqrt{\eta^2+|\nabla\nu|^2}} \dfrac{\nabla \nu^T }{\sqrt{\eta^2+|\nabla\nu|^2}} \nabla \nu \right)=\gamma\nabla \nu \left(1-(1+\mathcal{O}(\frac{\eta^2}{\|\nabla\nu\|^2}) )\right) \approx 0.$$

We can also see if the gradient of the reconstructed image and the reference image are perpendicular, $\nabla x^T\nabla \nu=0$, then

$$D\nabla x (I-\zeta \zeta^T)\nabla x= \nabla \nu - \dfrac{\nabla \nu }{\sqrt{\eta^2+|\nabla\nu|^2}} \dfrac{\nabla \nu^T }{\sqrt{\eta^2+|\nabla\nu|^2}} \nabla x =\nabla \nu $$

 and is non-zero. 

This regulariser encourages the gradient of the reconstructed image to be equal to or parallel to the gradient of the reference image. 

The CCPi regularisation toolkit implementation also allows us to add a non-negativity constraint. 



### Create reference image 

To use this regulariser we need a reference image. In the next cell we create a reference image using some of the spheres in the ground truth image. 

In [ ]:
#%% create masks
top = ig.allocate(0)
bottom = ig.allocate(0)
middle= ig.allocate(0)

top.fill(
    np.asarray(ground_truth.array > 0.8 * ground_truth.max(), 
               dtype=np.float32)
    )
bottom.fill(
    np.asarray(np.invert(ground_truth.array < 0.4 * ground_truth.max()), 
               dtype=np.float32)
)
middle.fill(
    np.asarray(np.invert(  ground_truth.array< 0.7 * ground_truth.max()), 
               dtype=np.float32)
)


reference=top*0.2+bottom*0.7+ middle*0.9
show2D([ground_truth, reference], title = ['Ground Truth', 'Reference'], origin = 'upper', num_cols = 2)

### Results using dTV
We can now compare the dTV results for a number of values of alpha. 

In [ ]:
for alpha in [1, 10, 100, 1000]:
    eta = 0.01
    F = LeastSquares(A = forward, b = noisy_absorption)
    G=FGP_dTV(reference=reference, alpha=alpha, eta=eta, nonnegativity=True)

    algo_dtv=FISTA(initial=ig.allocate(0), f=F, g=G, update_objective_interval=10)
    algo_dtv.run(250)
    show2D([ground_truth, recon_FDK, algo_tv.solution, reference,  algo_dtv.solution], title = ['Ground Truth', 'FDK Reconstruction', 'TV solution, alpha = 200', 'Reference image',  'dTV solution, alpha = {}'.format(alpha)], origin = 'upper', num_cols = 5, fix_range=[(0,0.004),(0,0.004),(0,0.004),(0,2),(0,0.004)])
    show2D([ground_truth, recon_FDK - ground_truth, algo_tv.solution - ground_truth, reference,  algo_dtv.solution - ground_truth], title = ['Ground Truth', 'FDK reconstruction error', 'TV solution error, alpha = 200', 'Reference image',  'dTV solution error, alpha = {}'.format(alpha)], origin = 'upper', num_cols = 5, fix_range=[(0,0.004),(-0.004,0.004),(-0.004,0.004),(0,2),(-0.004,0.004)], cmap=['gray', 'seismic', 'seismic', 'gray', 'seismic'],)




Too small a value of alpha and the solution is unstable because of the blurring operator, for intermediate values of alpha, the dTV reconstruction gives sharp edges around the spheres in the reference image and blurrier reconstructions of the other spheres. For too large values of alpha, the dTV solution fails to reconstruct the spheres not in the original image 

For a value of alpha that is too small we see evidence that the inverse problem is ill posed the noise has been amplified by the reconstruction

## Directional TV regularisation, written in CIL 

In this section we consider a denoising problem, denoising the FDK reconstruction using the directional TV regularisation term, i.e. solving the problem

$$ \arg \min_x \|x-x_{FDK}\|_2^2  + \alpha dTV(x) .$$ 

We use PDHG, which usually solves problems of the form 
$$
\min_{x\in\mathbb{X}} \mathcal{F}(K x) + \mathcal{G}(x)
\tag{PDHG form}
$$

where $F$ and $G$ need to have a calculable proximal and proximal conjugate,  respectively.  We choose $G$ to be the `L2NormSquared` function and build the $dTV$ function with an operator composed with the `MixedL21Norm` function which has a calculable proximal. 

In [ ]:
eta = 0.01
alpha = 0.01


# fidelity term
g = L2NormSquared( b=recon_FDK)

# setup operator for directional TV
DY = FiniteDifferenceOperator(ig, direction=1)
DX = FiniteDifferenceOperator(ig, direction=0)

Grad = BlockOperator(DY, DX)
grad_ref = Grad.direct(reference)
denom = (eta**2 + grad_ref.pnorm(2)**2).sqrt()
xi = grad_ref/denom

A1 = DY - CompositionOperator(DiagonalOperator(xi[0]**2),DY) - CompositionOperator(DiagonalOperator(xi[0]*xi[1]),DX)
A2 = DX - CompositionOperator(DiagonalOperator(xi[0]*xi[1]),DY) - CompositionOperator(DiagonalOperator(xi[1]**2),DX)

operator = BlockOperator(A1, A2)

f = alpha * MixedL21Norm()

# use primal acceleration, g being strongly convex
pdhg = PDHG(f = f, g = g, operator = operator,  update_objective_interval = 100, gamma_g = 1.)
pdhg.run(500)        


show2D([ground_truth, recon_FDK, algo_tv.solution, reference,  pdhg.solution], title = ['Ground Truth', 'FDK Reconstruction', 'TV solution, alpha = 200', 'Reference image',  'dTV solution'], origin = 'upper', num_cols = 5, fix_range=[(0,0.004),(0,0.004),(0,0.004),(0,2),(0,0.004)])
show2D([ground_truth, recon_FDK-ground_truth, algo_tv.solution-ground_truth, reference,  pdhg.solution-ground_truth], title = ['Ground Truth', 'FDK Reconstruction error', 'TV solution error, alpha = 200', 'Reference image',  'dTV solution error'], origin = 'upper', cmap=['gray', 'seismic', 'seismic', 'gray', 'seismic'], num_cols = 5, fix_range=[(0, 0.004),(-0.004, 0.004),(-0.004, 0.004),(0,2),(-0.008, 0.008)])


With less information, in the form of the forward model, the outcome is not as good, but arguably still better than the TV reconstruction. 

## Note 
This notebook is not quite complete. It would be good to use a CIL dTV implementation for the full forward problem. However, CIL currently can't take the proximal of an operator composed with a function so there is no proximal defined for the CIL LeastSquares function (https://github.com/TomographicImaging/CIL/issues/1561).